# Lab 02A: Agentic Workflow Patterns (Ollama Implementation)

This lab implements the five agentic workflow patterns described in Anthropic's engineering blog post [*Building Effective Agents*](https://www.anthropic.com/engineering/building-effective-agents).

All examples run **locally via Ollama** (zero API cost).

---

### Patterns covered

| # | Pattern | Core idea |
|---|---------|----------|
| 1 | **Prompt Chaining** | Sequential steps where each LLM call feeds the next |
| 2 | **Routing** | Classify input → dispatch to a specialised prompt/model |
| 3 | **Parallelization** | Run multiple LLM calls simultaneously, then aggregate |
| 4 | **Orchestrator-Workers** | Central LLM breaks down a task and delegates to workers |
| 5 | **Evaluator-Optimizer** | Generate → Evaluate → Refine loop |

> **Key insight from Anthropic:** Start simple. Only add complexity when it demonstrably improves outcomes.


## 0) Setup

Same environment as Lab 01. Make sure Ollama is running and your model is available.


In [ ]:
import os
import asyncio
import json
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

load_dotenv(override=True)

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
OLLAMA_MODEL    = os.getenv("OLLAMA_MODEL", "qwen3:8b")

client = OpenAI(
    base_url=f"{OLLAMA_BASE_URL.rstrip('/')}/v1",
    api_key=os.getenv("OLLAMA_API_KEY", "ollama"),
)

def chat(messages: list[dict], temperature: float = 0.7) -> str:
    """Thin wrapper: returns the assistant reply as a plain string."""
    response = client.chat.completions.create(
        model=OLLAMA_MODEL,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

print("Model:", OLLAMA_MODEL)
print("Ready ✓")


---
## Pattern 1 — Prompt Chaining

**Concept:** Decompose a complex task into a fixed sequence of smaller LLM calls. Each step's output becomes the next step's input. A *gate* (programmatic check) can be inserted between steps to catch problems early.

```
User Input → [Step 1 LLM] → output_1 → (gate?) → [Step 2 LLM] → output_2 → …
```

**Use when:** the task can be cleanly decomposed into sequential subtasks, and you want to trade latency for higher accuracy per step.

**Example below:** Blog post pipeline — outline → gate check → full draft → polish.


In [ ]:
TOPIC = "Why vector databases matter for RAG applications"

# ── Step 1: Generate an outline ──────────────────────────────────────────────
outline = chat([
    {"role": "system", "content": "You are a technical content strategist."},
    {"role": "user",   "content": f"Write a 3-section outline for a blog post about: {TOPIC}. Return only the outline."},
])
display(Markdown("### Step 1 — Outline\n\n" + outline))

# ── Gate: check the outline has exactly 3 sections ───────────────────────────
section_count = outline.count("\n#") + outline.count("\n1.") + outline.count("\n2.")
gate_pass = len(outline.strip()) > 50   # simple length guard
print(f"\n🔍 Gate check — outline non-empty: {gate_pass}")
assert gate_pass, "Outline too short — halting pipeline."

# ── Step 2: Write the draft ───────────────────────────────────────────────────
draft = chat([
    {"role": "system",    "content": "You are a senior technical writer."},
    {"role": "user",      "content": f"Topic: {TOPIC}"},
    {"role": "assistant", "content": outline},
    {"role": "user",      "content": "Expand the outline into a concise 200-word blog post draft."},
])
display(Markdown("### Step 2 — Draft\n\n" + draft))

# ── Step 3: Polish for a non-technical audience ───────────────────────────────
polished = chat([
    {"role": "system", "content": "You are an editor who simplifies technical writing."},
    {"role": "user",   "content": f"Rewrite the following blog post so it is engaging for a non-technical audience. Keep it under 150 words.\n\n{draft}"},
])
display(Markdown("### Step 3 — Polished\n\n" + polished))


---
## Pattern 2 — Routing

**Concept:** A *classifier* LLM (or rule) reads the input and assigns it a category. A second LLM call — with a prompt specialised for that category — handles the actual response.

```
User Input → [Classifier LLM] → category → [Specialist LLM for category] → Response
```

**Use when:** you have distinct input types that benefit from different prompts, tools, or even different models.

**Example below:** Customer support triage — billing, technical, or general queries each get a tailored response.


In [ ]:
SPECIALIST_PROMPTS = {
    "billing":   "You are a billing support specialist. Be empathetic and solution-focused.",
    "technical": "You are a senior technical support engineer. Give precise, step-by-step answers.",
    "general":   "You are a friendly customer success agent. Be warm and concise.",
}

def classify_query(user_query: str) -> str:
    """Return one of: billing | technical | general"""
    result = chat(
        [
            {"role": "system", "content": (
                "Classify the following customer query into exactly one category: "
                "billing, technical, or general. "
                "Reply with the single word only."
            )},
            {"role": "user", "content": user_query},
        ],
        temperature=0,
    ).strip().lower()
    # Normalise — model might add punctuation
    for key in SPECIALIST_PROMPTS:
        if key in result:
            return key
    return "general"

def routed_response(user_query: str) -> str:
    category = classify_query(user_query)
    system_prompt = SPECIALIST_PROMPTS[category]
    answer = chat([
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_query},
    ])
    return category, answer

# ── Test three query types ────────────────────────────────────────────────────
queries = [
    "I was charged twice on my last invoice.",
    "My API requests keep returning a 429 error even though I'm below the limit.",
    "What are your business hours?",
]

for q in queries:
    category, answer = routed_response(q)
    display(Markdown(f"**Query:** {q}\n\n**Routed to:** `{category}`\n\n**Response:** {answer}\n\n---"))


---
## Pattern 3 — Parallelization

**Concept:** Run multiple independent LLM calls at the same time and aggregate the results. Two sub-patterns:

- **Sectioning** — split a task into independent chunks (e.g. evaluate different aspects of code).
- **Voting** — run the same prompt N times and take a majority decision.

```
               ┌─ [Worker 1] ─┐
Input ─────────┤─ [Worker 2] ─├──► Aggregator ──► Final output
               └─ [Worker 3] ─┘
```

**Use when:** subtasks are independent, or you need higher confidence via multiple perspectives.

**Example below (Sectioning):** Evaluate a code snippet on three independent axes simultaneously — correctness, readability, security.


In [ ]:
CODE_SNIPPET = """
def get_user(user_id):
    query = f"SELECT * FROM users WHERE id = {user_id}"
    return db.execute(query).fetchone()
"""

REVIEW_AXES = {
    "correctness":  "Review the code only for logical correctness and potential bugs.",
    "readability":  "Review the code only for readability, naming, and documentation.",
    "security":     "Review the code only for security vulnerabilities.",
}

def review_axis(axis: str, instructions: str) -> tuple[str, str]:
    result = chat([
        {"role": "system", "content": instructions + " Be concise (3–5 sentences)."},
        {"role": "user",   "content": f"```python\n{CODE_SNIPPET}\n```"},
    ])
    return axis, result

# ── Run reviews in parallel using threads ─────────────────────────────────────
with ThreadPoolExecutor(max_workers=3) as pool:
    futures = {pool.submit(review_axis, axis, instr): axis
               for axis, instr in REVIEW_AXES.items()}
    reviews = {}
    for future in futures:
        axis, result = future.result()
        reviews[axis] = result

# ── Aggregation step ──────────────────────────────────────────────────────────
review_text = "\n\n".join(f"### {k.title()}\n{v}" for k, v in reviews.items())
display(Markdown("## Parallel Code Reviews\n\n" + review_text))

summary = chat([
    {"role": "system", "content": "You are a lead code reviewer. Summarise the findings below into a single priority-ordered action list."},
    {"role": "user",   "content": review_text},
])
display(Markdown("## Aggregated Action List\n\n" + summary))


### Parallelization — Voting variant

Run the same classification N times and take a majority vote. Useful when a single call may be unreliable.


In [ ]:
from collections import Counter

REVIEW_TEXT = """
The new product update is amazing! I love how smooth everything feels now.
Although the onboarding is still a bit confusing, the core experience is 10/10.
"""

def classify_sentiment(_: int) -> str:
    """Classify sentiment as positive, negative, or mixed."""
    return chat(
        [
            {"role": "system", "content": "Classify the sentiment as exactly one word: positive, negative, or mixed."},
            {"role": "user",   "content": REVIEW_TEXT},
        ],
        temperature=1,   # higher temp → more variance across runs
    ).strip().lower().split()[0]

N_VOTES = 5
with ThreadPoolExecutor(max_workers=N_VOTES) as pool:
    votes = list(pool.map(classify_sentiment, range(N_VOTES)))

tally = Counter(votes)
winner = tally.most_common(1)[0][0]
display(Markdown(f"**Votes:** {dict(tally)}\n\n**Majority verdict:** `{winner}`"))


---
## Pattern 4 — Orchestrator-Workers

**Concept:** A central *orchestrator* LLM receives the goal, breaks it into subtasks dynamically, and delegates each subtask to a *worker* LLM. The orchestrator then synthesises all worker outputs.

```
Goal ──► [Orchestrator LLM] ──► subtask_1 ──► [Worker 1]
                             ├─► subtask_2 ──► [Worker 2]   ──► [Orchestrator synthesises]
                             └─► subtask_3 ──► [Worker 3]
```

**Key difference from Parallelization:** subtasks are *not pre-defined*; the orchestrator decides them at runtime.

**Use when:** the task is complex and the required subtasks aren't known in advance.

**Example below:** Research assistant — the orchestrator decides which sub-questions to investigate, workers answer each, orchestrator synthesises a final report.


In [ ]:
RESEARCH_GOAL = "Explain how Retrieval-Augmented Generation (RAG) works and when to use it."

# ── Orchestrator: break into subtasks ─────────────────────────────────────────
plan_raw = chat(
    [
        {"role": "system", "content": (
            "You are a research orchestrator. Given a goal, output a JSON array of 3–4 "
            "focused sub-questions to investigate. Return ONLY valid JSON, no other text."
        )},
        {"role": "user", "content": RESEARCH_GOAL},
    ],
    temperature=0,
)

# Parse JSON (strip markdown fences if the model adds them)
plan_clean = plan_raw.strip().removeprefix("```json").removeprefix("```").removesuffix("```").strip()
subtasks: list[str] = json.loads(plan_clean)
display(Markdown("### Orchestrator Plan\n\n" + "\n".join(f"{i+1}. {t}" for i, t in enumerate(subtasks))))

# ── Workers: answer each subtask in parallel ──────────────────────────────────
def worker_answer(question: str) -> tuple[str, str]:
    answer = chat([
        {"role": "system", "content": "You are a concise AI/ML expert. Answer in 3–5 sentences."},
        {"role": "user",   "content": question},
    ])
    return question, answer

with ThreadPoolExecutor(max_workers=len(subtasks)) as pool:
    worker_results = dict(pool.map(lambda q: worker_answer(q), subtasks))

worker_text = "\n\n".join(f"**Q: {q}**\n\n{a}" for q, a in worker_results.items())
display(Markdown("### Worker Answers\n\n" + worker_text))

# ── Orchestrator: synthesise into a final report ──────────────────────────────
final_report = chat([
    {"role": "system", "content": "You are a senior technical writer. Synthesise the Q&A pairs below into a coherent, structured mini-report of ~200 words."},
    {"role": "user",   "content": f"Goal: {RESEARCH_GOAL}\n\n{worker_text}"},
])
display(Markdown("### Final Synthesised Report\n\n" + final_report))


---
## Pattern 5 — Evaluator-Optimizer

**Concept:** A *generator* LLM produces output; an *evaluator* LLM scores it and provides feedback; the generator uses that feedback to improve. The loop continues until quality criteria are met or a maximum iteration count is reached.

```
Task ──► [Generator] ──► draft
                ▲              │
           feedback      [Evaluator]
                ▲              │
                └──── score ◄──┘
           (loop until score ≥ threshold or max_iters)
```

**Use when:** you have clear quality criteria and iterative refinement measurably improves output. Analogous to a human writer → editor cycle.

**Example below:** Refine a technical explanation until the evaluator scores it ≥ 8/10 for clarity.


In [ ]:
WRITING_TASK = "Explain how attention mechanisms work in transformer models, for a junior developer."
SCORE_THRESHOLD = 8
MAX_ITERATIONS  = 4

def generate(task: str, feedback: str | None = None) -> str:
    messages = [
        {"role": "system", "content": "You are a technical educator. Write clearly and concisely (≤150 words)."},
        {"role": "user",   "content": task},
    ]
    if feedback:
        messages.append({"role": "user", "content": f"Improve your previous response based on this feedback:\n{feedback}"})
    return chat(messages)

def evaluate(task: str, response: str) -> tuple[int, str]:
    """Return (score 1-10, actionable feedback)."""
    raw = chat(
        [
            {"role": "system", "content": (
                "You are a strict writing evaluator. "
                "Score the response from 1–10 for clarity and accuracy given the task. "
                "Reply ONLY with JSON: {\"score\": <int>, \"feedback\": \"<string>\"}"
            )},
            {"role": "user", "content": f"Task: {task}\n\nResponse:\n{response}"},
        ],
        temperature=0,
    ).strip()
    clean = raw.removeprefix("```json").removeprefix("```").removesuffix("```").strip()
    parsed = json.loads(clean)
    return int(parsed["score"]), parsed["feedback"]

# ── Optimization loop ─────────────────────────────────────────────────────────
feedback = None
for iteration in range(1, MAX_ITERATIONS + 1):
    draft = generate(WRITING_TASK, feedback)
    score, feedback = evaluate(WRITING_TASK, draft)

    display(Markdown(
        f"### Iteration {iteration} — Score: {score}/10\n\n"
        f"**Draft:**\n\n{draft}\n\n"
        f"**Evaluator feedback:** {feedback}"
    ))

    if score >= SCORE_THRESHOLD:
        display(Markdown(f"✅ **Threshold reached at iteration {iteration}. Final answer above.**"))
        break
else:
    display(Markdown(f"⚠️ Max iterations reached. Best score: {score}/10"))


---
## Pattern Comparison

Run the same task through **Prompt Chaining** vs **Evaluator-Optimizer** and compare quality vs latency trade-offs.


In [ ]:
import time

TASK = "Write a 100-word explanation of embeddings for a product manager."

# ── Prompt Chaining: outline → draft (2 steps, no feedback loop) ──────────────
t0 = time.time()
outline_pm = chat([
    {"role": "system", "content": "You are a technical content strategist."},
    {"role": "user",   "content": f"Write a 3-point outline for: {TASK}"},
])
draft_pm = chat([
    {"role": "system",    "content": "You are a technical writer for business audiences."},
    {"role": "assistant", "content": outline_pm},
    {"role": "user",      "content": "Expand into a 100-word explanation."},
])
chain_time = time.time() - t0

# ── Evaluator-Optimizer: 1 generation + 1 refinement pass ─────────────────────
t0 = time.time()
draft_eo = generate(TASK)
score_eo, fb_eo = evaluate(TASK, draft_eo)
if score_eo < 8:
    draft_eo = generate(TASK, fb_eo)
eo_time = time.time() - t0

display(Markdown(
    f"## Prompt Chaining result ({chain_time:.1f}s)\n\n{draft_pm}\n\n---\n\n"
    f"## Evaluator-Optimizer result ({eo_time:.1f}s) — initial score {score_eo}/10\n\n{draft_eo}"
))


---
## Exercises

Work through as many as you like. Start with (1) and go deeper as time permits.

1. **Prompt Chaining - add retry + structured gate:** In the blog pipeline (Pattern 1), enforce a strict outline format (exactly 3 numbered sections with titles only). If the gate fails, automatically re-prompt the model up to 2 retries before halting. Print which attempt passed.

2. **Routing — add a fourth category:** Extend the routing example to handle `"feature_request"` queries. Write a specialist prompt and test it with at least two example inputs.

3. **Parallelization — build a guardrail:** Create a parallel two-worker setup where Worker A generates a response to a user question while Worker B simultaneously checks whether the question is safe/appropriate. Combine the two results — only return Worker A's output if Worker B gives the green light.

4. **Orchestrator-Workers — dynamic depth:** Modify the orchestrator example so that after the first round of worker answers, the orchestrator decides whether a second round of deeper sub-questions is warranted. Implement at most 2 rounds.

5. **Evaluator-Optimizer — custom rubric:** Change the evaluator prompt to score on three separate axes (clarity, accuracy, brevity) and return a JSON object with all three scores. Stop the loop only when *all three* scores are ≥ 7.

6. **(Stretch) Combine patterns:** Build a mini content pipeline that uses:
   - **Routing** to detect if the topic is technical or business-oriented,
   - **Orchestrator-Workers** to research the topic in parallel, and
   - **Evaluator-Optimizer** to refine the final output.


In [ ]:
# Your code here
